In [1]:
import pandas as pd
import os

# pd.set_option('display.max_rows', None)

In [2]:
asset_a_path = '/home/nick/Git/money-repo/lagged_Correlation/SPY_Hourly_AH.csv'
asset_b_path = '/home/nick/Git/money-repo/lagged_Correlation/SPY_Hourly_AH.csv'

In [3]:
asset_a_df = pd.read_csv(asset_a_path)
asset_b_df = pd.read_csv(asset_b_path)

In [4]:
high_thres = 0.2
low_thres = 0.2

In [5]:
lookback = 1140
lookahead = 60

In [6]:
# convert Date column to datetime
asset_a_df['Time'] = pd.to_datetime(asset_a_df['Time'])

# sort by date
asset_a_df.sort_values(by='Time', inplace=True)
asset_a_df.reset_index(drop=True, inplace=True)
asset_a_change = asset_a_df[['Time', 'Open']]

asset_b_df['Time'] = pd.to_datetime(asset_b_df['Time'])
asset_b_df.sort_values(by='Time', inplace=True)
asset_b_df.reset_index(drop=True, inplace=True)
asset_b_change = asset_b_df[['Time', 'Open']]

In [7]:
asset_a_change

,Time,Open
0,2016-01-04 04:00:00,201.1800
1,2016-01-04 05:00:00,200.8700
2,2016-01-04 06:00:00,201.0800
3,2016-01-04 07:00:00,200.2900
4,2016-01-04 08:00:00,200.5000
...,...,...
32967,2023-12-08 15:00:00,460.3800
32968,2023-12-08 16:00:00,460.1300
32969,2023-12-08 17:00:00,460.1897
32970,2023-12-08 18:00:00,460.3500


In [8]:
asset_a_change['Time']

0       2016-01-04 04:00:00
1       2016-01-04 05:00:00
2       2016-01-04 06:00:00
3       2016-01-04 07:00:00
4       2016-01-04 08:00:00
                ...        
32967   2023-12-08 15:00:00
32968   2023-12-08 16:00:00
32969   2023-12-08 17:00:00
32970   2023-12-08 18:00:00
32971   2023-12-08 19:00:00
Name: Time, Length: 32972, dtype: datetime64[ns]

In [9]:
times = {d: i for i, d in enumerate(asset_a_change['Time'])}

# make a new row of the Time column with the Time subtracted by lookback minutes
lookback_mask = asset_a_change['Time'] - pd.Timedelta(minutes=lookback)

# make a mask if the time is in Times. If it is, give the difference in rows, if not, give NaN
lookback_mask = lookback_mask.map(times.get).dropna()

lookback_mask.head(30)

16     5.0
17     6.0
18     7.0
19     8.0
20     9.0
21    10.0
22    11.0
23    12.0
24    13.0
25    14.0
26    15.0
32    21.0
33    22.0
34    23.0
35    24.0
36    25.0
37    26.0
38    27.0
39    28.0
40    29.0
41    30.0
42    31.0
48    37.0
49    38.0
50    39.0
51    40.0
52    41.0
53    42.0
54    43.0
55    44.0
Name: Time, dtype: float64

In [10]:
asset_a_change.head(30)

,Time,Open
0,2016-01-04 04:00:00,201.1800
1,2016-01-04 05:00:00,200.8700
2,2016-01-04 06:00:00,201.0800
3,2016-01-04 07:00:00,200.2900
4,2016-01-04 08:00:00,200.5000
5,2016-01-04 09:00:00,200.3500
6,2016-01-04 10:00:00,200.2200
7,2016-01-04 11:00:00,199.1500
8,2016-01-04 12:00:00,199.3900
9,2016-01-04 13:00:00,199.8000


In [11]:
# for the rows in lookback_mask, calculate the difference between the Open price of a time and the Open price of the time in lookback_mask
lookback_pct_change = pd.Series(index=asset_a_change.index, dtype='float64')
lookback_change = pd.Series(index=asset_a_change.index, dtype='float64')

for i in lookback_mask.index:
    lookback_pct_change[i] = (asset_a_change.loc[i, 'Open'] / asset_a_change.loc[lookback_mask[i], 'Open'] - 1) * 100
    lookback_change[i] = asset_a_change.loc[i, 'Open'] - asset_a_change.loc[lookback_mask[i], 'Open']

asset_a_change['Chg%'] = lookback_pct_change
asset_a_change['Chg'] = lookback_change

lookback_pct_change.head(30)

/tmp/ipykernel_10157/1038312956.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_a_change['Chg%'] = lookback_pct_change
/tmp/ipykernel_10157/1038312956.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_a_change['Chg'] = lookback_change


0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
5          NaN
6          NaN
7          NaN
8          NaN
9          NaN
10         NaN
11         NaN
12         NaN
13         NaN
14         NaN
15         NaN
16    0.394310
17    0.059934
18    0.527241
19    0.486484
20    0.065065
21    0.695313
22    0.661323
23    0.149254
24   -0.287203
25   -0.426884
26   -0.218510
27         NaN
28         NaN
29         NaN
dtype: float64

In [12]:
asset_a_change.head(30)

,Time,Open,Chg%,Chg
0,2016-01-04 04:00:00,201.1800,NaN,NaN
1,2016-01-04 05:00:00,200.8700,NaN,NaN
2,2016-01-04 06:00:00,201.0800,NaN,NaN
3,2016-01-04 07:00:00,200.2900,NaN,NaN
4,2016-01-04 08:00:00,200.5000,NaN,NaN
5,2016-01-04 09:00:00,200.3500,NaN,NaN
6,2016-01-04 10:00:00,200.2200,NaN,NaN
7,2016-01-04 11:00:00,199.1500,NaN,NaN
8,2016-01-04 12:00:00,199.3900,NaN,NaN
9,2016-01-04 13:00:00,199.8000,NaN,NaN


In [13]:
# split datetime into date and time
asset_a_change['Date'] = asset_a_change['Time'].dt.date
asset_a_change['Time'] = asset_a_change['Time'].dt.time

# convert both to datetime objects
asset_a_change['Date'] = pd.to_datetime(asset_a_change['Date'])
asset_a_change['Time'] = pd.to_datetime(asset_a_change['Time'], format='%H:%M:%S')

/tmp/ipykernel_10157/2151251943.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_a_change['Date'] = asset_a_change['Time'].dt.date
/tmp/ipykernel_10157/2151251943.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_a_change['Time'] = asset_a_change['Time'].dt.time
/tmp/ipykernel_10157/2151251943.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [14]:
asset_a_change.head(30)

,Time,Open,Chg%,Chg,Date
0,1900-01-01 04:00:00,201.1800,NaN,NaN,2016-01-04
1,1900-01-01 05:00:00,200.8700,NaN,NaN,2016-01-04
2,1900-01-01 06:00:00,201.0800,NaN,NaN,2016-01-04
3,1900-01-01 07:00:00,200.2900,NaN,NaN,2016-01-04
4,1900-01-01 08:00:00,200.5000,NaN,NaN,2016-01-04
5,1900-01-01 09:00:00,200.3500,NaN,NaN,2016-01-04
6,1900-01-01 10:00:00,200.2200,NaN,NaN,2016-01-04
7,1900-01-01 11:00:00,199.1500,NaN,NaN,2016-01-04
8,1900-01-01 12:00:00,199.3900,NaN,NaN,2016-01-04
9,1900-01-01 13:00:00,199.8000,NaN,NaN,2016-01-04


In [15]:
asset_a_df.head()

,Time,Open,High,Low,Last,Change,%Chg,Volume
0,2016-01-04 04:00:00,201.18,201.39,200.83,200.84,-3.23,-1.58%,40514
1,2016-01-04 05:00:00,200.87,201.13,200.75,201.11,0.27,0.13%,132231
2,2016-01-04 06:00:00,201.08,201.12,200.26,200.40,-0.71,-0.35%,266682
3,2016-01-04 07:00:00,200.29,200.65,199.98,200.50,0.10,0.05%,964268
4,2016-01-04 08:00:00,200.50,200.86,200.07,200.34,-0.16,-0.08%,1494608


In [16]:
asset_a_change.head(30)

,Time,Open,Chg%,Chg,Date
0,1900-01-01 04:00:00,201.1800,NaN,NaN,2016-01-04
1,1900-01-01 05:00:00,200.8700,NaN,NaN,2016-01-04
2,1900-01-01 06:00:00,201.0800,NaN,NaN,2016-01-04
3,1900-01-01 07:00:00,200.2900,NaN,NaN,2016-01-04
4,1900-01-01 08:00:00,200.5000,NaN,NaN,2016-01-04
5,1900-01-01 09:00:00,200.3500,NaN,NaN,2016-01-04
6,1900-01-01 10:00:00,200.2200,NaN,NaN,2016-01-04
7,1900-01-01 11:00:00,199.1500,NaN,NaN,2016-01-04
8,1900-01-01 12:00:00,199.3900,NaN,NaN,2016-01-04
9,1900-01-01 13:00:00,199.8000,NaN,NaN,2016-01-04


In [17]:
asset_a_change_pivot = asset_a_change.pivot(index='Date', columns='Time', values='Chg%')
# drop columns that are all NaN
asset_a_change_pivot = asset_a_change_pivot.dropna(axis=1, how='all')

# drop rows that are all NaN
asset_a_change_pivot = asset_a_change_pivot.dropna(axis=0, how='all')
asset_a_change_pivot = asset_a_change_pivot.dropna(axis=1, how='all')

asset_a_change_orig = asset_a_change.copy()
asset_a_change_pivot

Time,1900-01-01 04:00:00,1900-01-01 05:00:00,1900-01-01 06:00:00,1900-01-01 07:00:00,1900-01-01 08:00:00,1900-01-01 09:00:00,1900-01-01 10:00:00,1900-01-01 11:00:00,1900-01-01 12:00:00,1900-01-01 13:00:00,1900-01-01 14:00:00,1900-01-01 15:00:00
Date,,,,,,,,,,,,
2016-01-05,0.394310,0.059934,0.527241,0.486484,0.065065,0.695313,0.661323,0.149254,-0.287203,-0.426884,-0.218510,NaN
2016-01-06,-0.884252,-0.706749,-1.246895,-1.036134,-1.311067,-1.445284,-1.072812,-1.082262,-1.111332,-1.160829,-1.408940,NaN
2016-01-07,-2.071781,-2.354654,-2.459222,-2.302830,-2.173258,-1.746088,-0.793932,-0.809859,-1.501004,-1.786431,-2.486963,NaN
2016-01-08,0.501895,0.010195,-0.806329,-0.754294,-0.153280,0.730152,0.576310,0.154631,0.072169,0.103040,-0.077304,NaN
2016-01-12,-0.678932,0.291530,1.018277,0.577764,1.311870,1.971816,1.565609,0.515854,0.338771,-0.088579,-0.296196,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-01,0.070194,0.402021,0.378088,0.597153,0.317104,0.350282,0.221929,0.006574,0.499934,0.759952,0.486735,NaN
2023-12-05,-0.179647,-0.087723,0.112199,-0.271960,-0.400727,-0.188598,-0.114040,0.157663,-0.048216,0.004384,0.052570,NaN
2023-12-06,0.569193,0.452290,0.100571,0.306977,0.249874,0.515169,0.297821,0.133599,0.089843,-0.116126,-0.083178,NaN


In [18]:
asset_a_change_pivot = asset_a_change_pivot.applymap(lambda x: 1 if x >= high_thres else -1 if x <= -low_thres else pd.NaT)

# drop columns that are all NaN
asset_a_change_pivot = asset_a_change_pivot.dropna(axis=1, how='all')
asset_a_change_pivot = asset_a_change_pivot.dropna(axis=0, how='all')
asset_a_change_pivot = asset_a_change_pivot.dropna(axis=1, how='all')

asset_a_change_pivot

Time,1900-01-01 04:00:00,1900-01-01 05:00:00,1900-01-01 06:00:00,1900-01-01 07:00:00,1900-01-01 08:00:00,1900-01-01 09:00:00,1900-01-01 10:00:00,1900-01-01 11:00:00,1900-01-01 12:00:00,1900-01-01 13:00:00,1900-01-01 14:00:00,1900-01-01 15:00:00
Date,,,,,,,,,,,,
2016-01-05,1,NaT,1,1,NaT,1,1,NaT,-1,-1,-1,NaT
2016-01-06,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,NaT
2016-01-07,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,NaT
2016-01-08,1,NaT,-1,-1,NaT,1,1,NaT,NaT,NaT,NaT,NaT
2016-01-12,-1,1,1,1,1,1,1,1,1,NaT,-1,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-01,NaT,1,1,1,1,1,1,NaT,1,1,1,NaT
2023-12-05,NaT,NaT,NaT,-1,-1,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2023-12-06,1,1,NaT,1,1,1,1,NaT,NaT,NaT,NaT,NaT


In [19]:
asset_b_change

,Time,Open
0,2016-01-04 04:00:00,201.1800
1,2016-01-04 05:00:00,200.8700
2,2016-01-04 06:00:00,201.0800
3,2016-01-04 07:00:00,200.2900
4,2016-01-04 08:00:00,200.5000
...,...,...
32967,2023-12-08 15:00:00,460.3800
32968,2023-12-08 16:00:00,460.1300
32969,2023-12-08 17:00:00,460.1897
32970,2023-12-08 18:00:00,460.3500


In [20]:
times_b = {d: i for i, d in enumerate(asset_b_change['Time'])}

# make a new row of the Time column with the Time subtracted by lookback minutes
lookahead_mask = asset_b_change['Time'] + pd.Timedelta(minutes=lookahead)

# make a mask if the time is in Times. If it is, give the difference in rows, if not, give NaN
lookahead_mask = lookahead_mask.map(times_b.get).dropna()

In [21]:
# for the rows in lookback_mask, calculate the difference between the Open price of a time and the Open price of the time in lookback_mask
lookahead_pct_change = pd.Series(index=asset_b_change.index, dtype='float64')
lookahead_change = pd.Series(index=asset_b_change.index, dtype='float64')

for i in lookahead_mask.index:
    lookahead_pct_change[i] = (asset_b_change.loc[i, 'Open'] / asset_b_change.loc[lookahead_mask[i], 'Open'] - 1) * 100
    lookahead_change[i] = asset_b_change.loc[i, 'Open'] - asset_b_change.loc[lookahead_mask[i], 'Open']

asset_b_change['Chg%'] = lookahead_pct_change
asset_b_change['Chg'] = lookahead_change

/tmp/ipykernel_10157/395782470.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_b_change['Chg%'] = lookahead_pct_change
/tmp/ipykernel_10157/395782470.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_b_change['Chg'] = lookahead_change


In [22]:
asset_b_change

,Time,Open,Chg%,Chg
0,2016-01-04 04:00:00,201.1800,0.154329,0.3100
1,2016-01-04 05:00:00,200.8700,-0.104436,-0.2100
2,2016-01-04 06:00:00,201.0800,0.394428,0.7900
3,2016-01-04 07:00:00,200.2900,-0.104738,-0.2100
4,2016-01-04 08:00:00,200.5000,0.074869,0.1500
...,...,...,...,...
32967,2023-12-08 15:00:00,460.3800,0.054332,0.2500
32968,2023-12-08 16:00:00,460.1300,-0.012973,-0.0597
32969,2023-12-08 17:00:00,460.1897,-0.034821,-0.1603
32970,2023-12-08 18:00:00,460.3500,-0.021718,-0.1000


In [23]:
asset_b_change['Date'] = asset_b_change['Time'].dt.date
asset_b_change['Time'] = asset_b_change['Time'].dt.time

asset_b_change['Date'] = pd.to_datetime(asset_b_change['Date'])
asset_b_change['Time'] = pd.to_datetime(asset_b_change['Time'], format='%H:%M:%S')

/tmp/ipykernel_10157/2585082788.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_b_change['Date'] = asset_b_change['Time'].dt.date
/tmp/ipykernel_10157/2585082788.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_b_change['Time'] = asset_b_change['Time'].dt.time
/tmp/ipykernel_10157/2585082788.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [24]:
asset_b_change_pivot = asset_b_change.pivot(index='Date', columns='Time', values='Chg%')
asset_b_vals_pivot = asset_b_change.pivot(index='Date', columns='Time', values='Chg')

asset_b_change_pivot = asset_b_change_pivot.dropna(axis=1, how='all')
asset_b_change_pivot = asset_b_change_pivot.dropna(axis=0, how='all')
asset_b_change_pivot = asset_b_change_pivot.dropna(axis=1, how='all')

asset_b_change_orig = asset_b_change_pivot.copy()

asset_b_change_pivot = asset_b_change_pivot.applymap(lambda x: 1 if x > 0 else -1 if x <= 0 else pd.NaT)

asset_b_change_pivot = asset_b_change_pivot.dropna(axis=1, how='all')
asset_b_change_pivot = asset_b_change_pivot.dropna(axis=0, how='all')
asset_b_change_pivot = asset_b_change_pivot.dropna(axis=1, how='all')

In [25]:
asset_b_change_orig

Time,1900-01-01 04:00:00,1900-01-01 05:00:00,1900-01-01 06:00:00,1900-01-01 07:00:00,1900-01-01 08:00:00,1900-01-01 09:00:00,1900-01-01 10:00:00,1900-01-01 11:00:00,1900-01-01 12:00:00,1900-01-01 13:00:00,1900-01-01 14:00:00,1900-01-01 15:00:00,1900-01-01 16:00:00,1900-01-01 17:00:00,1900-01-01 18:00:00,1900-01-01 19:00:00
Date,,,,,,,,,,,,,,,,
2016-01-04,0.154329,-0.104436,0.394428,-0.104738,0.074869,0.064929,0.537283,-0.120367,-0.205205,-0.055025,0.155311,-0.696517,-0.228333,0.000000,-0.133842,NaN
2016-01-05,0.399321,0.069930,-0.079856,0.215075,-0.680576,0.189130,-0.188773,0.208382,0.140279,-0.342393,-0.025231,-0.044681,-0.064497,-0.009922,0.004961,NaN
2016-01-06,0.010025,0.357161,-0.005030,0.419255,-0.206674,-0.401647,-0.035132,-0.035119,0.040153,0.256630,0.495575,-0.528169,-0.200803,-0.040145,-0.080225,NaN
2016-01-07,-0.113116,0.072035,-0.195142,-0.092350,-0.179248,-0.468957,-0.512196,0.499465,0.250358,0.637598,0.072039,0.170094,0.010310,-0.056672,0.030921,NaN
2016-01-08,0.020387,0.306748,0.446772,-0.353086,-0.245023,0.225110,0.591838,0.092721,-0.087494,0.211460,0.279286,0.750352,0.031274,-0.052097,-0.078084,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-04,0.013100,0.032762,0.074315,0.013116,0.219082,0.103075,0.314597,-0.307051,-0.157663,0.171138,-0.019782,-0.151094,0.085474,0.015344,0.054764,NaN
2023-12-05,0.010975,0.114271,0.076972,-0.028581,-0.041755,-0.094410,-0.421959,0.291628,-0.037262,0.006576,-0.098544,0.013141,0.052591,-0.010955,-0.098501,NaN
2023-12-06,0.021857,-0.072076,0.085253,0.019678,-0.257376,0.117945,0.177165,0.096331,0.195231,-0.131443,0.140403,0.250720,-0.015393,0.046200,-0.013198,NaN


In [26]:
# For the two pivot tables, find the Date rows and Time columns that are in both tables
common_dates = asset_a_change_pivot.index.intersection(asset_b_change_pivot.index)
common_times = asset_a_change_pivot.columns.intersection(asset_b_change_pivot.columns)

# slice the pivot tables to only include the common dates and times
asset_a_change_pivot = asset_a_change_pivot.loc[common_dates, common_times]
asset_b_change_pivot = asset_b_change_pivot.loc[common_dates, common_times]

In [27]:
# set the rows and columns of asset_b_vals_pivot to match the rows and columns of asset_a_change_pivot
asset_b_vals_pivot = asset_b_vals_pivot.loc[common_dates, common_times]

In [28]:
asset_a_change_pivot

Time,1900-01-01 04:00:00,1900-01-01 05:00:00,1900-01-01 06:00:00,1900-01-01 07:00:00,1900-01-01 08:00:00,1900-01-01 09:00:00,1900-01-01 10:00:00,1900-01-01 11:00:00,1900-01-01 12:00:00,1900-01-01 13:00:00,1900-01-01 14:00:00,1900-01-01 15:00:00
Date,,,,,,,,,,,,
2016-01-05,1,NaT,1,1,NaT,1,1,NaT,-1,-1,-1,NaT
2016-01-06,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,NaT
2016-01-07,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,NaT
2016-01-08,1,NaT,-1,-1,NaT,1,1,NaT,NaT,NaT,NaT,NaT
2016-01-12,-1,1,1,1,1,1,1,1,1,NaT,-1,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-01,NaT,1,1,1,1,1,1,NaT,1,1,1,NaT
2023-12-05,NaT,NaT,NaT,-1,-1,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2023-12-06,1,1,NaT,1,1,1,1,NaT,NaT,NaT,NaT,NaT


In [29]:
asset_b_change_pivot

Time,1900-01-01 04:00:00,1900-01-01 05:00:00,1900-01-01 06:00:00,1900-01-01 07:00:00,1900-01-01 08:00:00,1900-01-01 09:00:00,1900-01-01 10:00:00,1900-01-01 11:00:00,1900-01-01 12:00:00,1900-01-01 13:00:00,1900-01-01 14:00:00,1900-01-01 15:00:00
Date,,,,,,,,,,,,
2016-01-05,1,1,-1,1,-1,1,-1,1,1,-1,-1,-1
2016-01-06,1,1,-1,1,-1,-1,-1,-1,1,1,1,-1
2016-01-07,-1,1,-1,-1,-1,-1,-1,1,1,1,1,1
2016-01-08,1,1,1,-1,-1,1,1,1,-1,1,1,1
2016-01-12,-1,-1,1,-1,-1,-1,1,1,1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-01,-1,1,1,1,1,-1,-1,-1,-1,1,-1,1
2023-12-05,1,1,1,-1,-1,-1,-1,1,-1,1,-1,1
2023-12-06,1,-1,1,1,-1,1,1,1,1,-1,1,1


In [30]:
probs_df = pd.DataFrame(columns = ['a_down_b_down',  'a_down_b_up', 'a_up_b_down', 'a_up_b_up'])
dist_df = pd.DataFrame(columns = ['a_down_b_down',  'a_down_b_up', 'a_up_b_down', 'a_up_b_up'])
trade_df = pd.DataFrame(columns = ['a_down_b_down',  'a_down_b_up', 'a_up_b_down', 'a_up_b_up'])

In [31]:
time_diff = 0

In [32]:
# for each column in asset_a_change_pivot
for col in asset_a_change_pivot.columns:
    asset_a_col = asset_a_change_pivot[col]

    # shift datetime.time by time_diff
    b_col = col + pd.Timedelta(minutes=time_diff)

    if b_col not in asset_b_change_pivot.columns:
        continue

    asset_b_col = asset_b_change_pivot[b_col]

    # get the combinations in tuple pairs, and any pairs with NaN are dropped
    combo_col = pd.DataFrame(list(zip(asset_a_col, asset_b_col))).dropna()

    asset_a_prob = combo_col[0].value_counts(normalize=True)
    conditional_probs = combo_col.groupby(0)[1].value_counts(normalize=True)

    conditional = conditional_probs.unstack().fillna(0).to_dict()

    full_conditional = {}

    full_conditional_stats = {}

    trade_stats = {}

    rename = {-1: "down", 1: "up"}

    for i in [-1, 1]:
        for j in [-1, 1]:
            if i not in conditional or j not in conditional[i]:
                full_conditional[f"a_{rename[j]}_b_{rename[i]}"] = 0
                full_conditional_stats[f"a_{rename[j]}_b_{rename[i]}"] = [0,0,0]
                trade_stats[f"a_{rename[j]}_b_{rename[i]}"] = []

            else:
                full_conditional[f"a_{rename[j]}_b_{rename[i]}"] = conditional[i][j]

                # get the rows for each combination of a and b
                a_down_b_down = combo_col[(combo_col[0] == j) & (combo_col[1] == i)]
                
                #print(asset_b_change_orig.iloc[a_down_b_down.index][b_col])
                # get the count, mean, and std from asset_b_vals_pivot for the rows in a_down_b_down for the hour
                a_down_b_down_vals = asset_b_change_orig.iloc[a_down_b_down.index][b_col].astype(float).describe().fillna(0).to_dict()
                #print(a_down_b_down_vals)
                full_conditional_stats[f"a_{rename[j]}_b_{rename[i]}"] = [int(a_down_b_down_vals['count']), a_down_b_down_vals['mean'], a_down_b_down_vals['std']]

                trade_history = asset_b_vals_pivot.iloc[a_down_b_down.index][b_col].astype(float).to_dict()
                
                # convert the dictionary to a list of tuples, converting Timestamp to string of only the date
                trade_history = [(str(k.date()), v) for k, v in trade_history.items()]

                trade_stats[f"a_{rename[j]}_b_{rename[i]}"] = trade_history

    name = col.strftime('%H:%M:%S')
    
    # # add the percentages to the dataframe using the name as the index
    probs_df.loc[name] = full_conditional
    dist_df.loc[name] = full_conditional_stats
    trade_df.loc[name] = trade_stats

In [33]:
# split the dates by year and generate the same dataframes as above for each year
years = asset_a_change_pivot.index.year.unique()

probs_df_yearly = {}
dist_df_yearly = {}
trade_df_yearly = {}

for year in years:
    asset_a_change_pivot_year = asset_a_change_pivot[asset_a_change_pivot.index.year == year]
    asset_b_change_pivot_year = asset_b_change_pivot[asset_b_change_pivot.index.year == year]
    asset_b_vals_pivot_year = asset_b_vals_pivot[asset_b_vals_pivot.index.year == year]

    probs_df_yearly[year] = pd.DataFrame(columns = ['a_down_b_down',  'a_down_b_up', 'a_up_b_down', 'a_up_b_up'])
    dist_df_yearly[year] = pd.DataFrame(columns = ['a_down_b_down',  'a_down_b_up', 'a_up_b_down', 'a_up_b_up'])
    trade_df_yearly[year] = pd.DataFrame(columns = ['a_down_b_down',  'a_down_b_up', 'a_up_b_down', 'a_up_b_up'])

    time_diff = 0

    # for each column in asset_a_change_pivot
    for col in asset_a_change_pivot_year.columns:
        asset_a_col = asset_a_change_pivot_year[col]

        # shift datetime.time by time_diff
        b_col = col + pd.Timedelta(minutes=time_diff)

        if b_col not in asset_b_change_pivot_year.columns:
            continue

        asset_b_col = asset_b_change_pivot_year[b_col]

        # get the combinations in tuple pairs, and any pairs with NaN are dropped
        combo_col = pd.DataFrame(list(zip(asset_a_col, asset_b_col))).dropna()

        asset_a_prob = combo_col[0].value_counts(normalize=True)
        conditional_probs = combo_col.groupby(0)[1].value_counts(normalize=True)

        conditional = conditional_probs.unstack().fillna(0).to_dict()

        full_conditional = {}

        full_conditional_stats = {}

        trade_stats = {}

        rename = {-1: "down", 1: "up"}

        for i in [-1, 1]:
            for j in [-1, 1]:
                if i not in conditional or j not in conditional[i]:
                    full_conditional[f"a_{rename[j]}_b_{rename[i]}"] = 0
                    full_conditional_stats[f"a_{rename[j]}_b_{rename[i]}"] = [0,0,0]
                    trade_stats[f"a_{rename[j]}_b_{rename[i]}"] = []

                else:
                    full_conditional[f"a_{rename[j]}_b_{rename[i]}"] = conditional[i][j]

                    # get the rows for each combination of a and b
                    a_down_b_down = combo_col[(combo_col[0] == j) & (combo_col[1] == i)]
                    
                    # get the count, mean, and std from asset_b_vals_pivot for the rows in a_down_b_down for the hour
                    a_down_b_down_vals = asset_b_change_orig.iloc[a_down_b_down.index][b_col].astype(float).describe().fillna(0).to_dict()

                    full_conditional_stats[f"a_{rename[j]}_b_{rename[i]}"] = [int(a_down_b_down_vals['count']), a_down_b_down_vals['mean'], a_down_b_down_vals['std']]

                    trade_history = asset_b_vals_pivot_year.iloc[a_down_b_down.index][b_col].astype(float).to_dict()
                    
                    # convert the dictionary to a list of tuples, converting Timestamp to string of only the date
                    trade_history = [(str(k.date()), v) for k, v in trade_history.items()]

                    trade_stats[f"a_{rename[j]}_b_{rename[i]}"] = trade_history

        name = col.strftime('%H:%M:%S')

        # add the percentages to the dataframe using the name as the index
        probs_df_yearly[year].loc[name] = full_conditional
        dist_df_yearly[year].loc[name] = full_conditional_stats
        trade_df_yearly[year].loc[name] = trade_stats
        
for year in years:
    print(year)
    print(probs_df_yearly[year].head())
    print("--------------------------")

2016
          a_down_b_down  a_down_b_up  a_up_b_down  a_up_b_up
04:00:00       0.468750     0.531250     0.602564   0.397436
05:00:00       0.650794     0.349206     0.573333   0.426667
06:00:00       0.629032     0.370968     0.441558   0.558442
07:00:00       0.555556     0.444444     0.473684   0.526316
08:00:00       0.533333     0.466667     0.484848   0.515152
--------------------------
2017
          a_down_b_down  a_down_b_up  a_up_b_down  a_up_b_up
04:00:00       0.511628     0.488372     0.500000   0.500000
05:00:00       0.567568     0.432432     0.481481   0.518519
06:00:00       0.625000     0.375000     0.464286   0.535714
07:00:00       0.628571     0.371429     0.542373   0.457627
08:00:00       0.692308     0.307692     0.551020   0.448980
--------------------------
2018
          a_down_b_down  a_down_b_up  a_up_b_down  a_up_b_up
04:00:00       0.453333     0.546667     0.552632   0.447368
05:00:00       0.533333     0.466667     0.493151   0.506849
06:00:00       0

In [34]:
probs_df

,a_down_b_down,a_down_b_up,a_up_b_down,a_up_b_up
04:00:00,0.522417,0.477583,0.548686,0.451314
05:00:00,0.543967,0.456033,0.546178,0.453822
06:00:00,0.604603,0.395397,0.488959,0.511041
07:00:00,0.542797,0.457203,0.517018,0.482982
08:00:00,0.520170,0.479830,0.469671,0.530329
09:00:00,0.533058,0.466942,0.519525,0.480475
10:00:00,0.525424,0.474576,0.505922,0.494078
11:00:00,0.546436,0.453564,0.577586,0.422414
12:00:00,0.545648,0.454352,0.559211,0.440789
13:00:00,0.520921,0.479079,0.541667,0.458333


In [35]:
dist_df

,a_down_b_down,a_down_b_up,a_up_b_down,a_up_b_up
04:00:00,"[266, -0.0077795948258136145, 0.17652183929869...","[245, 0.00015856132582585533, 0.18943890902695...","[354, 0.015251474317811478, 0.23924563897194023]","[291, -0.010353048857994799, 0.20221907058104627]"
05:00:00,"[266, -0.005363311566636651, 0.13757000162708996]","[222, -0.023275396922469594, 0.13412159342968283]","[342, -0.008448293883501533, 0.1568979496471399]","[284, -0.005058092964604285, 0.1873971039729116]"
06:00:00,"[289, 0.0023788405419556038, 0.17175374420292125]","[188, -0.0046488097582026945, 0.13560451325110...","[309, -0.01282806791532705, 0.295824492088717]","[323, -0.015313815349767487, 0.1967179056952917]"
07:00:00,"[260, -0.015907739128397165, 0.15304201910707219]","[219, -0.007180215906832174, 0.15121232095038106]","[318, 0.005992679662263631, 0.28934336090265894]","[298, 0.005234816705345046, 0.232787737102902]"
08:00:00,"[245, -0.030332145242809513, 0.2271421586931599]","[226, 0.008567477354079011, 0.22261098503726623]","[270, 0.0012034475885576896, 0.29106318223322186]","[306, 0.0011302700341977096, 0.4455336178228046]"
09:00:00,"[258, -0.03746533222107688, 0.3845610895882177]","[226, -0.0021058894283870707, 0.28609031397125...","[306, 0.005372004212212462, 0.41381037420376887]","[283, 0.008976458828623385, 0.2426882561083133]"
10:00:00,"[248, -0.0090141475691378, 0.29585948481496516]","[224, 0.028229828013237146, 0.35106738708735824]","[299, 0.02420181107104287, 0.39003194622512893]","[292, -0.019621164081965473, 0.3756440452622462]"
11:00:00,"[253, -0.010557475767844285, 0.23151343811027913]","[210, -0.01352070101046818, 0.27290213364049676]","[335, 0.001626928470826171, 0.32273699830227714]","[245, -0.03830136581826108, 0.2767557521394428]"
12:00:00,"[257, -0.008725206615944935, 0.19877561546233302]","[214, 0.018718322763457047, 0.23184609348919397]","[340, -0.030849692677425627, 0.32677384978128793]","[268, -0.0021657055638989254, 0.25981206500236..."
13:00:00,"[246, 0.008585817814787233, 0.20811947745083953]","[229, 0.02800952033248309, 0.26515259580163075]","[334, 0.01647560983159709, 0.26995975696251784]","[284, -0.0008607973265279064, 0.2627608989028555]"


In [36]:
trade_df

,a_down_b_down,a_down_b_up,a_up_b_down,a_up_b_up
04:00:00,"[(2016-01-07, -0.21999999999999886), (2016-01-...","[(2016-01-06, 0.020000000000010232), (2016-02-...","[(2016-01-22, -0.2600000000000193), (2016-01-2...","[(2016-01-05, 0.799999999999983), (2016-01-08,..."
05:00:00,"[(2016-01-26, -0.5099999999999909), (2016-02-1...","[(2016-01-06, 0.710000000000008), (2016-01-07,...","[(2016-01-12, -0.799999999999983), (2016-01-21...","[(2016-01-13, 0.06999999999999318), (2016-01-1..."
06:00:00,"[(2016-01-06, -0.010000000000019327), (2016-01...","[(2016-01-08, 0.8700000000000045), (2016-01-14...","[(2016-01-05, -0.160000000000025), (2016-01-22...","[(2016-01-12, 0.21999999999999886), (2016-01-1..."
07:00:00,"[(2016-01-07, -0.18000000000000682), (2016-01-...","[(2016-01-06, 0.8300000000000125), (2016-01-15...","[(2016-01-12, -0.6100000000000136), (2016-01-1...","[(2016-01-05, 0.4300000000000068), (2016-01-29..."
08:00:00,"[(2016-01-06, -0.4099999999999966), (2016-01-0...","[(2016-01-15, 1.0900000000000034), (2016-02-19...","[(2016-01-12, -0.09000000000000341), (2016-01-...","[(2016-01-13, 0.8799999999999955), (2016-01-22..."
09:00:00,"[(2016-01-06, -0.8000000000000114), (2016-01-0...","[(2016-01-14, 1.8400000000000034), (2016-01-20...","[(2016-01-12, -0.03999999999999204), (2016-01-...","[(2016-01-05, 0.3800000000000239), (2016-01-08..."
10:00:00,"[(2016-01-06, -0.06999999999999318), (2016-01-...","[(2016-01-20, 1.789999999999992), (2016-02-02,...","[(2016-01-05, -0.3800000000000239), (2016-01-2...","[(2016-01-08, 1.1500000000000057), (2016-01-12..."
11:00:00,"[(2016-01-06, -0.06999999999999318), (2016-01-...","[(2016-01-07, 0.9799999999999898), (2016-01-15...","[(2016-01-14, -0.09000000000000341), (2016-01-...","[(2016-01-12, 0.5407999999999902), (2016-01-22..."
12:00:00,"[(2016-01-20, -0.18000000000000682), (2016-01-...","[(2016-01-05, 0.2814000000000192), (2016-01-06...","[(2016-01-14, -0.1699999999999875), (2016-01-2...","[(2016-01-12, 0.7700000000000102), (2016-01-21..."
13:00:00,"[(2016-01-05, -0.6891999999999996), (2016-01-1...","[(2016-01-06, 0.5100000000000193), (2016-01-07...","[(2016-01-14, -1.1400000000000148), (2016-01-2...","[(2016-01-21, 0.8100999999999772), (2016-01-26..."
